## **ML Model 2**
## Predict Support Level based on Number of Editors

#### **German Support Tickets**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# PLEASE ADAPT PATH ACCORDINGLY
DATA_PATH  = '../../data/tickets_postprp.pkl'

df = pd.read_pickle(DATA_PATH)
df.info()


In [ ]:
df_german = df[df['language'] == 'de']

# for later
df_english = df[df['language'] == 'en']

df = df_german
df.head()

,kategorie_id,kategorietext,unterkategorie_id,unterkategorietext,status,angelegt_am,angelegt_von,auftraggeber,meldender,embedding,max_similarity,min_difference,faq_index_max_sim,faq_index_min_dif,num_editors,language
0,ZSD_SR_AA,Allgemeine Anfragen,ZSD_SR_AA_SON,Sonstiges,E0008 Quittiert,1.448477e+09,H2451TK1,H2451,H2451TK1,"[0.14102867, 0.87266093, 1.3615568, 0.51052284...",0.591029,9.864998,12,14,2,de
1,ZSD_SR_BO,SAP Netweaver Business Objects,SR_ZSD_BO_FP,Fallstudienproblem,E0008 Quittiert,1.448546e+09,H2741CR1,H2741,H2741CR1,"[-0.34037295, 1.1536975, 0.8353497, 0.52754015...",0.645721,9.467989,12,18,0,de
2,ZSD_SR_BO,SAP Netweaver Business Objects,SR_ZSD_BO_FP,Fallstudienproblem,E0008 Quittiert,1.448549e+09,H2741CR1,H2741,H2741CR1,"[-0.33647916, 1.0358676, 1.183024, 0.33042786,...",0.666763,7.828650,12,14,1,de
3,ZSD_SR_BYD,SAP Business by Design,ZSD_SR_BYD_PRM,Passwortrücksetzung für Masteruser,E0008 Quittiert,1.448560e+09,H2851GB2,H2851,H2851GB2,"[-0.1209006, 0.41539708, 1.6555126, 0.09115197...",0.552797,11.220666,12,17,1,de
4,ZSD_SR_GBI,SAP ERP GBI,ZSD_SR_GBI_SON,Sonstiges,E0008 Quittiert,1.448878e+09,H2702AR1,H2702,H2702AR1,"[-0.21527165, 1.3987509, 0.267509, 0.24152803,...",0.526619,9.676493,9,17,1,de


In [ ]:
# split the dataset
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.2, random_state=21, shuffle=True)

("train", train_data.shape), ("test" ,test_data.shape)

(('train', (7651, 16)), ('test', (1913, 16)))

In [ ]:
get_features_label = lambda df, label_name: (df.drop(label_name, axis=1), df[label_name])

train_X, train_y = get_features_label(train_data, "num_editors")
test_X, test_y = get_features_label(test_data, "num_editors")

print("train data", ("features", train_X.shape), ("target", train_y.shape))
print("test data", ("features", test_X.shape), ("target", test_y.shape))

train data ('features', (7651, 15)) ('target', (7651,))
test data ('features', (1913, 15)) ('target', (1913,))


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

# make sure to have these lists to select numerical and categorical features accordingly
categorical_features = ['kategorie_id', 'kategorietext', 'unterkategorie_id', 'unterkategorietext', 'angelegt_von', 'auftraggeber', 'meldender']


# cateogrical transformations (encoding)
cat_transformer = Pipeline([
        ('encoder', OneHotEncoder(handle_unknown = 'ignore')),])

# use the ColumnTransformer to map the transformations to the respective features
preprocessor = ColumnTransformer([
        ("categorical", cat_transformer, categorical_features),])

# fit and transform the data and store it in preprocessed_X
preprocessed_X = preprocessor.fit_transform(train_X)

In [ ]:
def fit_evaluate_model(model, preprocessor, train_X, train_y, test_X, test_y):

    # fit model to the preprocessed training data
    model.fit(preprocessed_X, train_y)

    # check and print performance on training data
    print("Training Accuracy: ", model.score(preprocessed_X, train_y))

    # transform test data using the preprocesser we have
    transformed_test_X = preprocessor.transform(test_X)

    # check and print performance on test data
    print("Test Accuracy: ", model.score(transformed_test_X, test_y))
    
    # print mean squared error on train and test
    print("Training Error: ", mean_squared_error(train_y, model.predict(preprocessed_X)))
    print("Testing Error: ", mean_squared_error(test_y, model.predict(transformed_test_X)))

    return model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
fit_evaluate_model(model, preprocessor, train_X, train_y, test_X, test_y)


Training Accuracy:  0.8023787740164684
Test Accuracy:  0.6811291165708312
Training Error:  0.27094497451313554
Testing Error:  0.47046523784631467


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
from sklearn import svm

model = svm.LinearSVC()
fit_evaluate_model(model, preprocessor, train_X, train_y, test_X, test_y)

Training Accuracy:  0.7502287282708143
Test Accuracy:  0.6816518557239938
Training Error:  0.34884328845902496
Testing Error:  0.46366962885520124


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

**Predict Support Level Based on Number of Editors**

The result of both models are  not very satisfying for German tickets.

#### **English Support Tickets**

As BERT was trained on English Wikipedia articles, we will now try to only use it for English support tickets

In [ ]:
df = df_english
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2515 entries, 7 to 12182
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   kategorie_id        2515 non-null   object 
 1   kategorietext       2515 non-null   object 
 2   unterkategorie_id   2515 non-null   object 
 3   unterkategorietext  2515 non-null   object 
 4   status              2515 non-null   object 
 5   angelegt_am         2515 non-null   float64
 6   angelegt_von        2515 non-null   object 
 7   auftraggeber        2515 non-null   object 
 8   meldender           2515 non-null   object 
 9   embedding           2515 non-null   object 
 10  max_similarity      2515 non-null   float64
 11  min_difference      2515 non-null   float64
 12  faq_index_max_sim   2515 non-null   int64  
 13  faq_index_min_dif   2515 non-null   int64  
 14  num_editors         2515 non-null   int64  
 15  language            2515 non-null   object 
dtypes: fl

In [ ]:
# split the dataset
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.2, random_state=21, shuffle=True)

("train", train_data.shape), ("test" ,test_data.shape)

(('train', (2012, 16)), ('test', (503, 16)))

In [ ]:
get_features_label = lambda df, label_name: (df.drop(label_name, axis=1), df[label_name])

train_X, train_y = get_features_label(train_data, "num_editors")
test_X, test_y = get_features_label(test_data, "num_editors")

print("train data", ("features", train_X.shape), ("target", train_y.shape))
print("test data", ("features", test_X.shape), ("target", test_y.shape))

train data ('features', (2012, 15)) ('target', (2012,))
test data ('features', (503, 15)) ('target', (503,))


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

# make sure to have these lists to select numerical and categorical features accordingly
categorical_features = ['kategorie_id', 'kategorietext', 'unterkategorie_id', 'unterkategorietext', 'angelegt_von', 'auftraggeber', 'meldender']


# cateogrical transformations (encoding)
cat_transformer = Pipeline([
        ('encoder', OneHotEncoder(handle_unknown = 'ignore')),])

# use the ColumnTransformer to map the transformations to the respective features
preprocessor = ColumnTransformer([
        ("categorical", cat_transformer, categorical_features),])

# fit and transform the data and store it in preprocessed_X
preprocessed_X = preprocessor.fit_transform(train_X)

In [ ]:
def fit_evaluate_model(model, preprocessor, train_X, train_y, test_X, test_y):

    # fit model to the preprocessed training data
    model.fit(preprocessed_X, train_y)

    # check and print performance on training data
    print("Training Accuracy: ", model.score(preprocessed_X, train_y))

    # transform test data using the preprocesser we have
    transformed_test_X = preprocessor.transform(test_X)

    # check and print performance on test data
    print("Test Accuracy: ", model.score(transformed_test_X, test_y))
    
    # print mean squared error on train and test  
    print("Training Error: ", mean_squared_error(train_y, model.predict(preprocessed_X)))
    print("Testing Error: ", mean_squared_error(test_y, model.predict(transformed_test_X)))

    return model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
fit_evaluate_model(model, preprocessor, train_X, train_y, test_X, test_y)

Training Accuracy:  0.7668986083499006
Test Accuracy:  0.49105367793240556
Training Error:  0.3543737574552684
Testing Error:  0.8369781312127237


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
from sklearn import svm

model = svm.LinearSVC()
fit_evaluate_model(model, preprocessor, train_X, train_y, test_X, test_y)

Training Accuracy:  0.6565606361829026
Test Accuracy:  0.5089463220675944
Training Error:  0.518389662027833
Testing Error:  0.8349900596421471


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

**Predict Support Level Based on Number of Editors**

The result of both models are even worse for English tickets. This migh be related to the fact, that the FAQs are only available in German. Hence, the similarity and distance features might not be that useful here.
